# Face Recognition
### 1) Import libraries
### 2) Import video
### 3) Convert to b/w
### 4) Extract frames
### 5) Mark the region of face
### 6) Create Custom Dataset for each person
### 7) Create Train and test loaders
### 8) Create CNN Algorithm
### 9) Test and improve accuracy
### 9) Identify real time image

After we create the b/w images, then we will pass the images through an algorithm which will give the position of face in the image, then we will crop(resize) the image to that shape and we will fix a certain shape so as to pad these images and make every image of the same size. These images will be the input to Siamese Network.

Reference video to import files from gdrive into colab: https://www.youtube.com/watch?v=BuuH0wsJ8-k

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from tqdm import tqdm
import glob

import random
import matplotlib.pyplot as plt
from PIL import Image
from itertools import combinations 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms

## Just Replace this path with the path to your video directory
video_dir = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos"

# if os.makedirs(path, exist_ok=True)
# Image_Dataset = pd.DataFrame(columns = ['label'] + [f'pixel_{i}' for i in range(600*600)])
# global Image_Dataset
# Triplet_Dataset = pd.DataFrame()
# Image_Dataset

In [50]:
person_name = OrderedDict()
person_name["Abhay Singh"] = 0
person_name["Abhishek Kumar"] = 1
person_name["Ajay Kumar"] = 2
person_name["Anand"] = 3
person_name["Ankit Soni"] = 4
person_name["Arjun Prajapat"] = 5
person_name["Arpit Jain"] = 6
person_name["Kushal Barochia"] = 7
person_name["Nakul Singh"] = 8
person_name["Ojas Lunia"] = 9
person_name["Pratik Wani"] = 10
person_name["Praveen"] = 11
person_name["Prince Kumar"] = 12
person_name["Raghav"] = 13
person_name["Rugved Kalaskar"] = 14
person_name["Shantanu"] = 15
person_name["Shrinath Mishra"] = 16
person_name["Siddharth Gupta"] = 17
person_name["Sohail"] = 18
person_name["Tejas Mathurkar"] = 19
person_name["Venkatesh"] = 20
person_name["Vikash"] = 21
person_name["Vivek"] = 22
person_name["Vrukshal Balki"] = 23
person_name["Yash Gupta"] = 24
OrderedDict()

TypeError: 'int' object is not iterable

##  Function to extract faces from video, crop them to same size and save them in a different folder
Try adding data by columns instead of by rows

In [45]:
# Function to extract images from video, and save just the faces from all the images in a seperate folder
def video_to_faces(video_path, label,img_size):
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return 0,0
    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # We can use os.chdir(path) to change the working directory
#     arr = np.array([])
    width, height =  video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    df = pd.DataFrame(columns = np.arange(int(width)*int(height)))
    face_classifier = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    max_h, max_w = 600,600
#     path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
    path = video_path.replace('Videos', 'Images')
    path = path.replace('.mp4', '')
    os.makedirs(path, exist_ok=True)  # Ensure the directory exists or create it
    for i in range(0,int(total_frames),10):
        frame_id = i
        file_path = os.path.join(path , f'{frame_id}.jpg')
        if os.path.exists(file_path):
            continue
        else:
            video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
            ret, frame = video.read()
            if not ret:
                print(f"Error reading frame {frame_id}")
                continue
            gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face = face_classifier.detectMultiScale(
                gray_image, scaleFactor=1.2, minNeighbors=9, minSize=(250, 250), maxSize=(max_h  ,max_w))
#             x, y, w, h = 0,0,0,0
    #         print(face)
            if len(face) == 0: ## If no face is detected then it will skip this frame
                continue
            for (x, y, w, h) in face:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
                x1 = abs(max_h//2 - x - w//2)
                x2 = abs(max_h//2 + x + w//2)
                y1 = abs(max_w//2 - y - h//2)
                y2 = abs(max_w//2 + y + h//2)
                im_crop = gray_image[y1:y2, x1:x2]
                im_crop = cv2.resize(im_crop, (img_size, img_size), interpolation=cv2.INTER_AREA) # try using 100*100
#                 flat_img = im_crop.flatten()
#                 print(flat_img)
#                 Modified the following code so that the image gets flattened and we will add the flattened image along with its label in. new rules. stop.
#                 new_row = pd.DataFrame([[label] + flat_img.tolist()])
#                 Image_Dataset = pd.concat([Image_Dataset, new_row], ignore_index=True)
    #         im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2RGB)
#             im_crop = cv2.cvtColor(im_crop, cv2.COLOR_GRAY2RGB)
#             import matplotlib.pyplot as plt
#             plt.figure(figsize=(10,10))
#             plt.imshow(im_crop, cmap = 'gray')
#             plt.axis('off')
    #         path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
#             print(im_crop.shape)
                cv2.imwrite(file_path, im_crop)
#     print("done")
    return 
# video_to_faces('Abhay Singh.mp4', "Abhay singh")
# video_to_faces('C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos\\Abhay Singh.mp4', "Abhay Singh")
# print(max_h, max_w)
plt.close()

## Code to convert all the videos to face images and store them in seperate folders according to their names

In [47]:
## Code to be used later to get a list of sub-folders in a folder
## folders = [x[0] for x in os.walk(video_dir)]
img_size = 150
# Image_Dataset = pd.DataFrame(columns = ['label'] + [f'pixel_{i}' for i in range(img_size*img_size)])
videos_list = [f for f in os.listdir(video_dir) if f.endswith(('.mp4'))]
for video in tqdm(videos_list):
#     print("Working on", video) 
    video_path = video_dir + "\\" + str(video)
    label = video.replace('.mp4','')
    video_to_faces(video_path, label,img_size)
#     max_h, max_w = max(h, max_h), max(w, max_w)
#     print(max_h, max_w)
#     Image_Dataset.to_csv('Image_Dataset.csv')
print("Done")

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.92s/it]

Done


## Generate Random Triplets

In [48]:
triplets_data = pd.DataFrame() 
## add all the created triplets to a Dataframe by flattening them by using df.loc[i] and save it to an excel file
def generate_random_triplets(images_path, pairs_per_class):
    '''
    We will choose the number of triplets based on the number of images in each folder.
    Lets say we will choose at max 8 pairs of anchor and positive images from a folder.
    If a folder has less than 5 images then we will skip it.
    '''
    triplets_path = images_path.replace("Images", "Triplets")
    os.makedirs(triplets_path, exist_ok=True)
    anchor = []
    pos = []
    neg = []
    folders = [x[0] for x in os.walk(images_path)]
    folders = folders[1:]
    for folder in tqdm(folders):
#         print(folders)
        path = os.path.join(images_path, folder)
        image_files = [f for f in os.listdir(path) if f.endswith('.jpg')]
#         print("Image_files", image_files)
        if not image_files:
            continue
        if len(image_files) < 4:
            #Skip folder with less than 8 images
            continue
        for i in range(pairs_per_class):
            current_path = os.path.join(triplets_path, str(f'{folders.index(folder)}_{i}'))
            os.makedirs(current_path, exist_ok=True)
            ## Choose anchor and positive images
            # Choose 2 random numbers to be the indexes for anchor and pos image
            anchor_idx, positive_idx = np.random.choice(len(image_files), size=2, replace=False)
#             print(anchor_idx, )
            anchor_image = cv2.imread(os.path.join(path, image_files[anchor_idx]))
#             anchor_image = Image.open(os.path.join(images_path, image_files[anchor_idx]))
#             print("anchor", image_files[anchor_idx])
            pos_image = cv2.imread(os.path.join(path, image_files[positive_idx]))
#             print('pos', pos_image)
            # For neg image we will randomly choose a folder and randomly choose an image from it
            neg_index = np.random.choice([x for x in range(0, len(folders)) if x != folder]) 
            neg_folder = os.path.join(images_path, folders[neg_index])
            neg_image_files = [f for f in os.listdir(neg_folder) if f.endswith(('.jpg'))]
            if not neg_image_files:
                continue
            neg_image = cv2.imread(os.path.join(neg_folder, np.random.choice(neg_image_files)))
#             print('neg shape', neg_image.shape)
#             print(folders[neg_index], len(neg_image_files))
#             if len(neg_image_files) > 4:
#                 neg_img = np.random.randint(0, len(neg_image_files))
#                 path_neg = os.path.join(neg_folder, neg_image_files[neg_img])
#                 img = cv2.imread(path_neg)
#                 neg.append(img)
#             fig = plt.figure(figsize=(10,7))
#             rows, cols = 1,3
#             fig.add_subplot(rows, cols, 1)
#             plt.imshow(anchor_image, cmap = 'gray')
#             plt.axis('off')
#             plt.title('Anchor Image')
#             fig.add_subplot(rows, cols, 1)
#             plt.imshow(pos_image)
#             plt.axis('off')
#             plt.title('Positive Image')
#             fig.add_subplot(rows, cols, 1)
#             plt.imshow(neg_image)
#             plt.axis('off')
#             plt.title('Negative Image')
#             anchor_image = cv2.imread(os.path.join(images_path, image_files[anchor_idx]))
#             print("Anchor Image Shape:", anchor_image.shape)  # Add this line to check the shape
#             pos_image = cv2.imread(os.path.join(images_path, image_files[positive_idx]))
#             print("Positive Image Shape:", pos_image.shape)  # Add this line to check the shape
#             neg_image = cv2.imread(os.path.join(neg_folder, np.random.choice(neg_image_files)))
#             print("Negative Image Shape:", neg_image.shape) 
            cv2.imwrite(os.path.join(current_path, "anchor.jpg"),anchor_image)
            cv2.imwrite(os.path.join(current_path, "pos.jpg"),pos_image)
            cv2.imwrite(os.path.join(current_path, "neg.jpg"),neg_image)
            anchor.append(anchor_image)
            pos.append(pos_image)
            neg.append(neg_image)
    return anchor, pos, neg
image_dir = video_dir.replace("Videos", "Images")
anchor, pos, neg = generate_random_triplets(image_dir,8)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:03<00:00,  7.56it/s]


(150, 150, 3)


AttributeError: 'list' object has no attribute 'shape'

## Triplet loss function

In [177]:
def triplet_loss(anchor, positive, negative, margin=0.2):
    pos_dist = (anchor - positive).pow(2).sum(-1) #.pow(.5)
    neg_dist = (anchor - negative).pow(2).sum(-1) #.pow(.5)
    loss = F.relu(pos_dist - neg_dist + margin)
    return loss.mean()

## Custom Dataset Class for the triplets

In [181]:
class Dataset(Dataset):
    def __init__(self, triplet_folder, transform = transforms.ToTensor()):
        self.transform = transform
        folders = [x[0] for x in os.walk(images_path)]
        self.folders = folders[1:]
#         os.makedirs(current_path, exist_ok=True)
        path = os.path.join(images_path, folder)
        
#         self.anchor = torch.from_numpy(anchor)
#         self.pos = torch.from_numpy(anchor)
#         self.neg = torch.from_numpy(neg)
    def __len__(self):
        return len(self.folders)
    
    def __getitem__(self, idx):
        folder = folders[idx]
        current_path = os.path.join(triplet_folder, str(folders.index(folder)))
        image_files = [f for f in os.listdir(path) if f.endswith('.jpg')]
        
        
        return self.anchor[idx], self.pos[idx], self.neg[idx]
triplet_folder = video_dir.replace("Videos", "Triplets")
Triplet_Dataset = Dataset(triplet_folder)

TypeError: list indices must be integers or slices, not tuple

In [ ]:
## Split Dataset into train and test

train_size = int(0.9 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(Triplet_Dataset, [train_size, test_size])

## CNN Code

# Code to Open Webcam, draw a rectangle around face and label it


In [196]:
# access video from the webcam
video_capture = cv2.VideoCapture(0)
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    x, y, w, h = 0,0,0,0
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
        x, y, w, h = x, y, w, h
    return faces, x, y
while True:
    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully
    faces, x, y = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame
    title = 'Face'
    cv2.putText(video_frame,title , (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (30,255,30), 2)
    cv2.imshow(
        "My Face Detection Project", video_frame
    )
    # display the processed frame in a window named "My Face Detection Project"
    # add label to the face
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
video_capture.release()
cv2.destroyAllWindows()